# Lecture 5

 Fall 2023

A demonstration of advanced `pandas` syntax to accompany Lecture 5.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

## More on `Groupby`

### Slido Exercise

Try to predict the results of the `groupby` operation shown. The answer is below the image.

<img src="/content/drive/MyDrive/groupby.png" alt="Image" width="600">

The top ?? will be "hi", the second ?? will be "tx", and the third ?? will be "sd".

In [3]:
# Form a data frame using dictionary
# Answer Here
ds = pd.DataFrame(dict(x=[3, 1, 4, 1, 5, 9, 2, 5, 6],
                      y=['ak', 'tx', 'fl', 'hi', 'mi', 'ak', 'ca', 'sd', 'nc']),
                      index=list('ABCABCACB') )
ds

,x,y
A,3,ak
B,1,tx
C,4,fl
A,1,hi
B,5,mi
C,9,ak
A,2,ca
C,5,sd
B,6,nc


In [ ]:
#Use groupby on index and get max of each group


### Loading `babynames` Dataset

In [ ]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "/content/drive/MyDrive/data/babynamesbystate.zip"
if not os.path.exists(local_filename): # If the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'STATE.CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.tail(10)

### Case Study: Name "Popularity"

In this exercise, let's find the name with sex "F" that has dropped most in popularity since its peak usage in California. We'll start by filtering `babynames` to only include names corresponding to sex "F".

In [ ]:
# Select the names only. of baby grils


In [ ]:
# We sort the data by year


To build our intuition on how to answer our research question, let's visualize the prevalence of the name "Jennifer" over time.

In [ ]:
# We'll talk about how to generate plots in a later lecture


We'll need a mathematical definition for the change in popularity of a name in California.

Define the metric "Ratio to Peak" (RTP). We'll calculate this as the count of the name in 2022 (the most recent year for which we have data) divided by the largest count of this name in *any* year.

A demo calculation for Jennifer:

In [ ]:
# In the year with the highest Jennifer count, 6065 Jennifers were born


6065

In [ ]:
# Remember that we sorted f_babynames by year.
# This means that grabbing the final entry gives us the most recent count of Jennifers: 114
# In 2022, the most recent year for which we have data, 114 Jennifers were born


114

In [ ]:
# Compute the RTP


0.018796372629843364

We can also write a function that produces the `ratio_to_peak`for a given `Series`. This will allow us to use `.groupby` to speed up our computation for all names in the dataset.

In [ ]:
def ratio_to_peak(series):
    """
    Compute the RTP for a Series containing the counts per year for a single name
    """


In [ ]:
# Construct a Series containing our Jennifer count data

# Then, find the RTP


0.018796372629843364

Now, let's use `.groupby` to compute the RTPs for *all* names in the dataset.

You may see a warning message when running the cell below. As discussed in the lecture, `pandas` can't apply an aggregation function to non-numeric data (it doens't make sense to divide "CA" by a number). We can select numerical columns of interest directly.

In [ ]:
rtp_table = f_babynames.groupby("Name")[["Year", "Count"]].agg(ratio_to_peak)
rtp_table

In [ ]:
# Results in a TypeError
# rtp_table = f_babynames.groupby("Name").agg(ratio_to_peak)
# rtp_table

### Slido Exercise

Is there a row where `Year` is not equal to 1?

In [ ]:
# Find Unique values in the Year column of rtp_table dataframe


In [ ]:
# Dropping the Year column


In [ ]:
# Rename "Count" to "Count RTP" for clarity


In [ ]:
# What name has fallen the most in popularity?


We can visualize the decrease in the popularity of the name "Debra:"

In [ ]:
def plot_name(*names):
    fig = px.line(f_babynames[f_babynames["Name"].isin(names)],
                  x = "Year", y = "Count", color="Name",
                  title=f"Popularity for: {names}")
    fig.update_layout(font_size = 18,
                  width=1000,
                  height=400)
    return fig

plot_name("Debra")

In [ ]:
# Find the 10 names that have decreased the most in popularity


In [ ]:
plot_name(*top10)

For fun, try plotting your name or your friends' names.

### Slido Exercise

Given the example below on `babynames` dataset, write code to compute the total number of babies with each name in California using with and without agg.

In [ ]:
# code here

### Slido Exercise

Write code to compute the total number of babies born each year in California.

In [ ]:
# code here

In [ ]:
# Plotting baby counts per year


### `groupby.size` and `groupby.count()`

In [ ]:
df = pd.DataFrame({'letter':['A', 'A', 'B', 'C', 'C', 'C'],
                   'num':[1, 2, 3, 4, np.NaN, 4],
                   'state':[np.NaN, 'tx', 'fl', 'hi', np.NaN, 'ak']})
df

`groupby.size()` returns a `Series`, indexed by the `letter`s that we grouped by, with values denoting the number of rows in each group/sub-DataFrame. It does not care about missing (`NaN`) values.

In [ ]:
# Use groupby with size()


letter
A    2
B    1
C    3
dtype: int64

`groupby.count()` returns a `DataFrame`, indexed by the `letter`s that we grouped by. Each column represents the number of non-missing values for that `letter`.

In [ ]:
# Use groupby with count()


,num,state
letter,,
A,2,1
B,1,1
C,2,2


You might recall `value_counts()` function we talked about last week. What's the difference?

In [ ]:
# Use value_count() on DataFrame described above


C    3
A    2
B    1
Name: letter, dtype: int64

Turns out `value_counts()` does something similar to `groupby.size()`, except that it also sorts the resulting `Series` in descending order.

## Filtering by Group

In [ ]:
# Let's read the elections dataset


Let's keep only the elections years where the maximum vote share `%` is less than 45%.

In [ ]:
# use filter function


,Candidate,Party,%,Year,Result
7,Clinton,Democratic,43.0,1992,win
8,Bush,Republican,37.4,1992,loss
9,Perot,Independent,18.9,1992,loss


### `groupby` Puzzle

Assume that we want to know the best election by each party.

#### Attempt #1

We have to be careful when using aggregation functions. For example, the code below might be misinterpreted to say that Woodrow Wilson successfully ran for election in 2020. Why is this happening?

In [ ]:
# Use agg(max)


,Candidate,%,Year,Result
Party,,,,
Democratic,Obama,52.9,2016,win
Independent,Perot,18.9,1996,loss
Republican,Trump,58.8,2016,win


#### Attempt #2

Next, we'll write code that properly returns _the best result by each party_. That is, each row should show the Year, Candidate, Popular Vote, Result, and % for the election in which that party saw its best results (rather than mixing them as in the example above). Here's what the first rows of the correct output should look like:

![parties.png](attachment:ab21f8de-ad29-46c2-bea7-e9aea9c40e3e.png)

In [ ]:
elections_sorted_by_percent.groupby("Party").first()

#### Alternative Solutions

You'll soon discover that with Pandas rich tool set, there's typically more than one way to get to the same answer. Each approach has different tradeoffs in terms of readability, performance, memory consumption, complexity, and more. It will take some experience for you to develop a sense of which approach is better for each problem, but you should, in general, try to think if you can at least envision a different solution to a given problem, especially if you find your current solution to be particularly convoluted or hard to read.

Here are a couple of other ways of obtaining the same result (in each case, we only show the top part with `head()`). The first approach uses `groupby` but finds the location of the maximum value via the `idxmax()` method (look up its documentation!).  We then index and sort by `Party` to match the requested formatting:

In [ ]:
# Use idxmax function


Party
Democratic     17
Independent     9
Republican      3
Name: %, dtype: int64

In [ ]:
# This is the computational part, Extract DataFrame based on above mentioned condition


# Apply sort_index to match the formatting above


,Candidate,%,Year,Result
Party,,,,
Democratic,Obama,52.9,2008,win
Independent,Perot,18.9,1992,loss
Republican,Reagan,58.8,1984,win


Another approach is listed below. And this one doesn't even use `groupby`!

This approach instead uses the `drop_duplicates` method to keep only the last occurrence of of each party after having sorted by "%", which is the best performance.

In [ ]:
# code here

,Candidate,%,Year,Result
Party,,,,
Democratic,Obama,52.9,2008,win
Independent,Perot,18.9,1992,loss
Republican,Reagan,58.8,1984,win


*Challenge:* See if you can find yet another approach that still gives the same answer.

### `DataFrameGroupBy` Objects

The result of `groupby` is not a `DataFrame` or a list of `DataFrame`s. It is instead a special type called a `DataFrameGroupBy`.

In [ ]:
grouped_by_party = elections.groupby("Party")
type(grouped_by_party)
elections

`GroupBy` objects are structured like dictionaries. In fact, we can actually see the dictionaries with the following code:

In [ ]:
# visualize groups

The `key`s of the dictionary are the groups (in this case, `Party`), and the `value`s are the **indices** of rows belonging to that group. We can access a particular sub-`DataFrame` using `get_group`:

In [ ]:
# code here

---